In [24]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Overview

### Protecting Gen AI Applications from Data Leakage
Large language models (LLMs) are deep learning models trained on massive datasets of text. LLMs can translate language, summarize text, generate creative writing, generate code, power chatbots and virtual assistants, and complement search engines and recommendation systems.


When incorporating your own data into generative AI applications especially via [Model Tuning](https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-models) it's possible to return a prediction with data from your dataset. In this case many organizations may want to filter LLM responses for sensitive data.


By using Sensitive Data Protection(Cloud DLP) we can identify and take corrective action on sensitive data within LLM responses in real time.


### Using Sensitive Data Protection(Cloud DLP) to filter LLM Responses

[Sensitive Data Protection(Cloud DLP)](https://cloud.google.com/dlp) is a fully managed service designed to discover, classify, and protect your sensitive data, where it resides from databases, text-based content, or even images. It uses a variety of methods to identify sensitive data, including regular expressions, dictionaries, and contextual elements. Once sensitive data is identified, Sensitive Data Protection(Cloud DLP) can take several actions to either classify it, mask it, encrypt it or even delete it.


Sensitive Data Protection(Cloud DLP) can be accessed via Cloud Console and used to scan data within Cloud Storage, BigQuery and other Google Cloud services. The following notebook demonstrates using it through the SDK to incorporate Sensitive Data Protection(Cloud DLP) capabilities directly into you Generative AI enabled applications


### Objectives

In this tutorial, you will learn how to use Sensitive Data Protection(Cloud DLP) API with the Python SDK and explore how to identify and redact sensitive data within a response from PaLM 2 LLM

By the end of the notebook, you should be able to understand various configurations of Sensitive Data Protection(Cloud DLP) like `inspect_config`, `deidentify_config`, `item`, and what each variable controls.

The steps performed include:

- Installing the Python SDKs
- Understand a Data Leakage scenario
  - Text generation model with `text-bison@001`
    - Understanding prompt manipulation to return sensitive data
- Understand Data Leakage Mitigations
  - Using Sensitive Data Protection(Cloud DLP) with `text-bison@001` responses
  

### Costs
This tutorial uses billable components of Google Cloud:

* Vertex AI Generative AI Studio
* Sensitive Data Protection(Cloud DLP)

Learn about pricing for [Vertex AI](https://cloud.google.com/vertex-ai/pricing), and
 [Sensitive Data Protection(Cloud DLP)](https://cloud.google.com/dlp/pricing). Use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Data governance and security
For more information, see the documentation on [Data Governance and Generative AI](https://cloud.google.com/vertex-ai/docs/generative-ai/data-governance) on Google Cloud.

### Responsible AI
Large language models (LLMs) can translate language, summarize text, generate creative writing, generate code, power chatbots and virtual assistants, and complement search engines and recommendation systems. At the same time, as an early-stage technology, its evolving capabilities and uses create potential for misapplication, misuse, and unintended or unforeseen consequences. Large language models can generate output that you don't expect, including text that's offensive, insensitive, or factually incorrect.

What's more, the incredible versatility of LLMs is also what makes it difficult to predict exactly what kinds of unintended or unforeseen outputs they might produce. Given these risks and complexities, the PaLM API is designed with [Google's AI Principles](https://ai.google/principles/) in mind. However, it is important for developers to understand and test their models to deploy safely and responsibly. To aid developers, the Generative AI Studio has built-in content filtering, and the PaLM API has safety attribute scoring to help customers test Google's safety filters and define confidence thresholds that are right for their use case and business. Please refer to the [Safety filters and attributes](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/responsible-ai#safety_filters_and_attributes) section to learn more.

When the PaLM API is integrated into a customer's unique use case and context, additional responsible AI considerations and [PaLM limitations](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/responsible-ai#palm_limitations) may need to be considered. We encourage customers to leverage fairness, interpretability, privacy and security [recommended practices](https://ai.google/responsibilities/responsible-ai-practices/).

## Getting Started

Install the necessary libraries

In [28]:
#@title Install Vertex AI and Sensitive Data Protection SDK
# Install Google Cloud Vertex AI
!pip install google-cloud-aiplatform --upgrade --user
# Install DLP
! pip install google-cloud-dlp



### If using Colab - Restart Runtime
After installing the necessary Python SDKs you must restart the python runtime. There are a few options based on environment:

**Colab**:
1. Click the "Restart Runtime" button in the output of the SDK installs
2. Click "Runtime" on the top toolbar -> Click "Restart Runtime"
3. Run Colab Runtime Restart Code Block



In [29]:
#@title Colab Runtime Restart
#import os
#os.kill(os.getpid(), 9)

Copy & Paste the GCP project id in the PROJECT_ID variabile bellow:

In [30]:
#@title Set Project and Location
PROJECT_ID = "qwiklabs-gcp-01-5519ec46ed75"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
! gcloud config set project {PROJECT_ID}


Updated property [core/project].


### Authenticating your notebook environment
* If you are using **Colab** to run this notebook, uncomment the cell below and continue. 
* If you are using **Vertex AI Worknebch** you can skip the next step (authenticate_user)

In [31]:

# Used for Colab, skipped if running on Vertex-AI Workbench
#from google.colab import auth
#auth.authenticate_user(project_id=PROJECT_ID)


### Set Model for Prediction/Generation (text-bison@001)

Import the TextGenerationModel library, this library is a part of the Vertex AI SDK for Python. It provides a simple way to interact with the text generation capabilities of large language models hosted on GCP.

In [40]:
from datetime import datetime
import json
import time

from google.cloud import storage
from vertexai.batch_prediction import BatchPredictionJob
from vertexai.generative_models import GenerativeModel

MODEL_ID = "gemini-1.5-flash"  # @param {type:"string", isTemplate: true}

model = GenerativeModel(MODEL_ID)

## Understanding and Mitigating Data Leakage

Let's ask a naive questions:

In [44]:
#@title Threat Use Case: Extract Sensitive Data & Data Leakage Scenario
#model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content("Write a story about a magic backpack.")
print(response.text)

Barnaby had always been a dreamer. He dreamt of faraway lands, of talking animals, and of adventures that would make his heart soar. But Barnaby lived in a small, dusty town where the only adventures were finding a lost sock in the laundry or arguing with the shopkeeper about the price of bread. 

One day, while rummaging through his grandmother's attic, Barnaby stumbled upon a dusty leather backpack tucked away in a forgotten corner. Its straps were worn, and the leather was cracked, but there was a glint in the old brass buckle that caught his eye. He pulled it out, the musty scent of age filling his nostrils, and as he unlatched the buckle, a rush of cool air whooshed past him. 

Inside, nestled between faded maps and a worn compass, lay a small, silver book. He opened it, and the pages seemed to come alive with vibrant colours and strange symbols. He flipped through the pages, mesmerized, until his eyes landed on a picture of a magnificent, shimmering city, bathed in moonlight. Bel

We want to block any PII data in the prompt response, to do so we will use Google Cloud __[Senstive Data Protection (SDP)] (https://cloud.google.com/security/products/dlp)__, a.k.a known as DLP. 
Sensitive Data Protection is a suite of services designed to help you discover, classify, and protect sensitive data within your GCP environment and beyond. It provides tools and capabilities to manage data risks and meet compliance requirements.

Key Uses of Sensitive Data Protection:

Data Discovery and Classification:

> It helps identify sensitive data like personally identifiable information (PII), financial data, or intellectual property across various GCP services (BigQuery, Cloud Storage, Datastore).

> It uses predefined and customizable data patterns (InfoTypes) to recognize different types of sensitive information.

Data Risk Analysis:

> Sensitive Data Protection integrates with Security Command Center to provide a comprehensive view of data risks by considering vulnerabilities, threat exposure, and the sensitivity of the data.

> It enables you to prioritize security efforts based on the potential impact of data breaches.

Data De-identification:

> It offers de-identification techniques like masking, tokenization, and bucketing to protect sensitive data while preserving its utility for analysis or processing.

> This allows you to share or use data without exposing sensitive details.

Data Loss Prevention (DLP):

> Sensitive Data Protection includes Cloud DLP, which enables real-time scanning of data in motion or at rest to detect and prevent unauthorized access or exfiltration.

> You can define policies to automatically redact or block sensitive information.

**InfoTypes** are predefined or custom categories that define specific types of sensitive information. They act as identifiers for the types of data that the service should look for and potentially protect. More details about InfoTypes [here] (https://cloud.google.com/sensitive-data-protection/docs/concepts-infotypes).

In the example below we will use two InfoTypes: "PERSON_NAME" and "EMAIL_ADDRESS", list of all supported InfoTypes are avaliable [here] (https://cloud.google.com/sensitive-data-protection/docs/infotypes-reference), SDP users can also create [custom InfoTypes] (https://cloud.google.com/sensitive-data-protection/docs/creating-custom-infotypes).



In [45]:
#@title Inspect and Redact PaLM 2 output with Sensitive Data Protection(Cloud DLP)

import google.cloud.dlp  # noqa: F811, E402
from typing import List  # noqa: F811, E402

def deidentify_with_replace_infotype(
    project: str, item: str, info_types: List[str]
) -> None:
    """Uses the Data Loss Prevention API to deidentify sensitive data in a
    string by replacing it with the info type.
    Args:
        project: The Google Cloud project id to use as a parent resource.
        item: The string to deidentify (will be treated as text).
        info_types: A list of strings representing info types to look for.
            A full list of info type categories can be fetched from the API.
    Returns:
        None; the response from the API is printed to the terminal.
    """

    # Instantiate a client
    dlp = google.cloud.dlp_v2.DlpServiceClient()

    # Convert the project id into a full resource id.
    parent = f"projects/{PROJECT_ID}"

    # Construct inspect configuration dictionary
    inspect_config = {"info_types": [{"name": info_type} for info_type in info_types]}

    # Construct deidentify configuration dictionary
    deidentify_config = {
        "info_type_transformations": {
            "transformations": [
                {"primitive_transformation": {"replace_with_info_type_config": {}}}
            ]
        }
    }

    # Call the API
    response = dlp.deidentify_content(
        request={
            "parent": parent,
            "deidentify_config": deidentify_config,
            "inspect_config": inspect_config,
            "item": {"value": item},
        }
    )

    # Print out the results.
    print(response.item.value)

deidentify_with_replace_infotype(PROJECT_ID, response.text, ["PERSON_NAME","EMAIL_ADDRESS"])

[PERSON_NAME] had always been a dreamer. He dreamt of faraway lands, of talking animals, and of adventures that would make his heart soar. But [PERSON_NAME] lived in a small, dusty town where the only adventures were finding a lost sock in the laundry or arguing with the shopkeeper about the price of bread. 

One day, while rummaging through his grandmother's attic, [PERSON_NAME] stumbled upon a dusty leather backpack tucked away in a forgotten corner. Its straps were worn, and the leather was cracked, but there was a glint in the old brass buckle that caught his eye. He pulled it out, the musty scent of age filling his nostrils, and as he unlatched the buckle, a rush of cool air whooshed past him. 

Inside, nestled between faded maps and a worn compass, lay a small, silver book. He opened it, and the pages seemed to come alive with vibrant colours and strange symbols. He flipped through the pages, mesmerized, until his eyes landed on a picture of a magnificent, shimmering city, bathed

The sensitive content in the output was de-identified by replacing the sensitive data with the type of data, e.g.: EMAIL. SDP supports multiple transformation options, such as reduction, masking and more, [this page] (https://cloud.google.com/sensitive-data-protection/docs/transformations-reference) lists all available transformation options. 

Challenge:

In some cases, a country when used to reference someone's place of birth, residence, or citizenship is considered senstive information.
Given the prompt below, mask the country name.



In [48]:
prompt = f"""Which nation is Albert Einstein's birthplace?
  """
response = model.generate_content(prompt)
#generation_model.predict(prompt)
response



candidates {
  content {
    role: "model"
    parts {
      text: "Albert Einstein was born in **Germany**. \n"
    }
  }
  avg_logprobs: -0.27571859359741213
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.219726562
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.176757812
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.25
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.101074219
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.249023438
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.163085938
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.0927734375
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0981445312
  }
}
model_version: "gemini

In [49]:
#@title Inspect and Redact PaLM 2 output with Sensitive Data Protection(Cloud DLP)

import google.cloud.dlp  # noqa: F811, E402
from typing import List  # noqa: F811, E402

def deidentify_with_replace_infotype(
    project: str, item: str, info_types: List[str]
) -> None:
    """Uses the Data Loss Prevention API to deidentify sensitive data in a
    string by replacing it with the info type.
    Args:
        project: The Google Cloud project id to use as a parent resource.
        item: The string to deidentify (will be treated as text).
        info_types: A list of strings representing info types to look for.
            A full list of info type categories can be fetched from the API.
    Returns:
        None; the response from the API is printed to the terminal.
    """

    # Instantiate a client
    dlp = google.cloud.dlp_v2.DlpServiceClient()

    # Convert the project id into a full resource id.
    parent = f"projects/{PROJECT_ID}"

    # Construct inspect configuration dictionary
    inspect_config = {"info_types": [{"name": info_type} for info_type in info_types]}

    # Construct deidentify configuration dictionary
    deidentify_config = {
        "info_type_transformations": {
            "transformations": [
                {"primitive_transformation": {"replace_with_info_type_config": {}}}
            ]
        }
    }

    # Call the API
    response = dlp.deidentify_content(
        request={
            "parent": parent,
            "deidentify_config": deidentify_config,
            "inspect_config": inspect_config,
            "item": {"value": item},
        }
    )

    # Print out the results.
    print(response.item.value)

deidentify_with_replace_infotype(PROJECT_ID, response.text, ["PERSON_NAME","LOCATION"])

[PERSON_NAME] was born in **[LOCATION]**. 

